In [1]:
import pandas as pd
import numpy as np

import miditoolkit

import re
import glob
import os

from music21 import *
import os
import pandas as pd
import glob

from tqdm import tqdm

from utils.utils import open_midi, get_file_name, extract_tempo, extract_key, to_snake_case

In [2]:
p_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/program_change.csv')
com_df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/commu_meta.csv', index_col=0)
df = pd.read_csv(r'/Users/hai/Desktop/hch/projects/midi/csv/allgenre_cp.csv')

In [3]:
commu_folder = './commu_midi'
com_m = glob.glob(f'{commu_folder}/*.mid')
com_m.sort()

In [4]:
our_m = list()
for idx, v in enumerate(df['song_title'].values):

    genre = df['genre'].values[idx]
    data_folder = './data/' + genre + '/cropped'

    our_m.append(os.path.join(data_folder, v + '/' + df['file_name'].values[idx]))

In [5]:
 chord_lst = [
        "a",
        "a#",
        "b",
        "c",
        "c#",
        "d",
        "d#",
        "e",
        "f",
        "f#",
        "g",
        "g#",
        "ab",
        "bb",
        "db",
        "eb",
        "gb",
    ]

In [6]:
def sync_key_augment(chords, aug_key, origin_key):
    chord_lst = [
        "a",
        "a#",
        "b",
        "c",
        "c#",
        "d",
        "d#",
        "e",
        "f",
        "f#",
        "g",
        "g#",
        "ab",
        "bb",
        "db",
        "eb",
        "gb",
    ]
    chord2symbol = {k: v for k, v in zip(chord_lst, range(12))}
    chord2symbol["ab"] = 11
    chord2symbol["bb"] = 1
    chord2symbol["db"] = 4
    chord2symbol["eb"] = 6
    chord2symbol["gb"] = 9
    symbol2chord = {v: k for k, v in chord2symbol.items()}

    basic_chord = []
    for c in chords:
        match = re.match(r"[A-G](#|b|)", c)
        basic_chord.append(match[0])

    chord_type = [c.replace(b, "") for c, b in zip(chords, basic_chord)]
    symbol_lst = [chord2symbol[c.lower()] for c in basic_chord]

    origin_key_symbol = chord2symbol[origin_key]

    augment_key_symbol = chord2symbol[aug_key]

    key_diff = origin_key_symbol - augment_key_symbol
    key_change = abs(key_diff)
    if key_diff < 0:
        new_symbol_lst = []
        for s in symbol_lst:
            new_s = s + key_change
            if new_s >= 12:
                new_s = new_s - 12
            new_symbol_lst.append(new_s)
    else:
        new_symbol_lst = []
        for s in symbol_lst:
            new_s = s - key_change
            if new_s < 0:
                new_s = new_s + 12
            new_symbol_lst.append(new_s)

    new_chord_lst = [symbol2chord[s] for s in new_symbol_lst]
    new_chord_lst = [c + t for c, t in zip(new_chord_lst, chord_type)]
    return [new_chord_lst]

In [7]:
score = converter.parse(our_m[60396])

In [8]:
score = converter.parse(com_m[9999])

In [9]:
com_df.iloc[9999]

audio_key                                                        aminor
chord_progressions    [['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'A...
pitch_range                                                     mid_low
num_measures                                                          8
bpm                                                                  90
genre                                                         cinematic
track_role                                                          pad
inst                                                  string_ensemble-2
sample_rhythm                                                  standard
time_signature                                                      4/4
min_velocity                                                        127
max_velocity                                                        127
split_data                                                        train
id                                                           com

In [10]:
com_df['chord_progressions'][9999]

"[['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm']]"

In [11]:
com_c = ['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 
  'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 
  'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 
  'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 
  'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 
  'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 
  'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 'Am7', 
  'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm', 'Dm']

In [12]:
len(com_c)

64

In [13]:
names = list()

for obj in score.recurse():
    if isinstance(obj, note.Note):
        names.append(obj.nameWithOctave)
    elif isinstance(obj, chord.Chord):
        c = obj.root().name +'_'+ obj.commonName
        names.append(to_snake_case(c))

In [14]:
note_list = list()
for obj in score.measures(2,2).recurse():
    if isinstance(obj, note.Note):
        note_list.append(obj.name)
    elif isinstance(obj, chord.Chord):
        for p in obj.pitches:
            note_list.append(p.name)

In [15]:
current_chord = chord.Chord(set(note_list))

In [16]:
new_list = list()
for n in note_list:
    p_c = pitch.Pitch(n)
    new_list.append(p_c.pitchClass)

In [17]:
new_list

[9, 5]

In [18]:
new_list_c = chord.Chord(set(note_list))
new_list_c

<music21.chord.Chord F A>

In [19]:
new_list_c.commonName

'Major Third'

In [20]:
c = new_list_c.root().name +'_'+ new_list_c.commonName

In [22]:
to_snake_case(c)

'f_major_third'

In [58]:
names

['a_perfect_fifth',
 'f_minor_sixth',
 'a_minor_seventh',
 'f_minor_sixth',
 'a_perfect_fifth',
 'f_minor_sixth',
 'a_minor_seventh',
 'f_minor_sixth']

In [48]:
chords = list(chords.replace('[', '').replace(']', '').replace("'", '').split(', '))

In [54]:
chords[-4]

'minor_second_above_b'

In [59]:
basic_chord = []
for c in names:
    match = re.match(r"[a-g](#|b|)", c)
    print(match)
    basic_chord.append(match[0])

<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='f'>
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='f'>
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='f'>
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='f'>


In [64]:
basic_chord

['a', 'f', 'a', 'f', 'a', 'f', 'a', 'f']

In [65]:
chord2symbol = {k: v for k, v in zip(chord_lst, range(12))}
chord2symbol["ab"] = 11
chord2symbol["bb"] = 1
chord2symbol["db"] = 4
chord2symbol["eb"] = 6
chord2symbol["gb"] = 9
symbol2chord = {v: k for k, v in chord2symbol.items()}

In [66]:
chord_type = [c.replace(b, "") for c, b in zip(names, basic_chord)]
symbol_lst = [chord2symbol[c.lower()] for c in basic_chord]


In [67]:
chord_type

['_perfect_fifth',
 '_minor_sixth',
 '_minor_seventh',
 '_minor_sixth',
 '_perfect_fifth',
 '_minor_sixth',
 '_minor_seventh',
 '_minor_sixth']